In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import io
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
df1 = pd.read_excel('D:/KPMG/KPMG_data.xlsx', sheet_name='CustomerDemographic', usecols=['customer_id','first_name','last_name','gender','past_3_years_bike_related_purchases','DOB','AGE','Age_Group','job_title','job_industry_category','wealth_segment','deceased_indicator','owns_car','tenure','address','postcode','state','country','property_valuation','Customer Segment'], engine='openpyxl')
df1 = df1[df1['customer_id'] <= 3500]
df1 = df1.dropna(subset=['Age_Group'])
df1.tail()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,AGE,Age_Group,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Customer Segment
3495,3496.0,Danya,Burnyeat,M,99.0,1986-04-25,34.971682,31-40,Editor,Manufacturing,Mass Customer,N,Yes,19.0,2565 Caliangt Point,2171.0,NSW,Australia,9.0,Evasive Customer
3496,3497.0,Thia,O'Day,F,73.0,1986-05-03,34.949764,31-40,Administrative Assistant IV,Manufacturing,Affluent Customer,N,Yes,18.0,96 Delladonna Trail,3976.0,VIC,Australia,5.0,Almost Lost Customer
3497,3498.0,Lois,Abrahim,F,28.0,1995-11-02,25.442915,21-30,NaN,Manufacturing,Mass Customer,N,No,5.0,3 Nova Point,3012.0,VIC,Australia,4.0,Evasive Customer
3498,3499.0,Shelton,Tewkesberrie,M,29.0,1979-06-17,41.831956,41-50,NaN,Manufacturing,Mass Customer,N,Yes,7.0,310 Stephen Terrace,4073.0,QLD,Australia,9.0,Losing Customer
3499,3500.0,Josy,Fleeman,F,71.0,1967-07-21,53.747024,51-60,NaN,Entertainment,Affluent Customer,N,No,17.0,9491 Green Ridge Terrace,2100.0,NSW,Australia,10.0,Evasive Customer


In [4]:
print(df1.columns.ravel())

['customer_id' 'first_name' 'last_name' 'gender'
 'past_3_years_bike_related_purchases' 'DOB' 'AGE' 'Age_Group' 'job_title'
 'job_industry_category' 'wealth_segment' 'deceased_indicator' 'owns_car'
 'tenure' 'address' 'postcode' 'state' 'country' 'property_valuation'
 'Customer Segment']


In [5]:
df1_new = df1.drop(columns=['customer_id','first_name','last_name','DOB','AGE','deceased_indicator','tenure','address','postcode','country'])
df1_new.head()

,gender,past_3_years_bike_related_purchases,Age_Group,job_title,job_industry_category,wealth_segment,owns_car,state,property_valuation,Customer Segment
0,F,93.0,61-70,Executive Secretary,Health,Mass Customer,Yes,NSW,10.0,Very Loyal
1,M,81.0,41-50,Administrative Officer,Financial Services,Mass Customer,Yes,NSW,10.0,Evasive Customer
2,M,61.0,61-70,Recruiting Manager,Property,Mass Customer,Yes,NSW,10.0,Almost Lost Customer
3,M,33.0,51-60,NaN,IT,Mass Customer,No,QLD,9.0,Lost Customer
4,F,56.0,41-50,Senior Editor,NaN,Affluent Customer,Yes,NSW,4.0,Becoming Loyal


In [6]:
df1_new.dtypes

gender                                  object
past_3_years_bike_related_purchases    float64
Age_Group                               object
job_title                               object
job_industry_category                   object
wealth_segment                          object
owns_car                                object
state                                   object
property_valuation                     float64
Customer Segment                        object
dtype: object

In [7]:
df2 = pd.read_excel('D:/KPMG/KPMG_data.xlsx', sheet_name='NewCustomerList', usecols=['first_name','last_name','gender','past_3_years_bike_related_purchases','DOB','AGE','Age_Group','job_title','job_industry_category','wealth_segment','deceased_indicator','owns_car','tenure','address','postcode','state','country','property_valuation'], engine='openpyxl')
#df2 = df2.dropna(subset=['Age_Group'])
df2.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,AGE,Age_Group,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation
0,Chickie,Brister,M,86,1957-07-12,63.777161,61-70,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6
1,Morly,Genery,M,69,1970-03-22,51.075791,51-60,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11
2,Ardelis,Forrester,F,10,1974-08-28,46.637435,41-50,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5
3,Lucine,Stutt,F,64,1979-01-28,42.215517,41-50,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1
4,Melinda,Hadlee,F,34,1965-09-21,55.577161,51-60,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9


In [8]:
df2_new = df2.drop(columns=['first_name','last_name','DOB','AGE','deceased_indicator','tenure','address','postcode','country'])
df2_new.head()

,gender,past_3_years_bike_related_purchases,Age_Group,job_title,job_industry_category,wealth_segment,owns_car,state,property_valuation
0,M,86,61-70,General Manager,Manufacturing,Mass Customer,Yes,QLD,6
1,M,69,51-60,Structural Engineer,Property,Mass Customer,No,NSW,11
2,F,10,41-50,Senior Cost Accountant,Financial Services,Affluent Customer,No,VIC,5
3,F,64,41-50,Account Representative III,Manufacturing,Affluent Customer,Yes,QLD,1
4,F,34,51-60,Financial Analyst,Financial Services,Affluent Customer,No,NSW,9


In [9]:
df2_new.dtypes

gender                                 object
past_3_years_bike_related_purchases     int64
Age_Group                              object
job_title                              object
job_industry_category                  object
wealth_segment                         object
owns_car                               object
state                                  object
property_valuation                      int64
dtype: object

In [10]:
X_train = df1_new.iloc[:,0:9]
y_train = df1_new.iloc[:,9]
X_test = df2_new.iloc[:,0:9]

In [11]:
# Scale numeric values
num_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# One-hot encode categorical values
obj_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('vect', CountVectorizer(ngram_range=(2,2)))])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, selector(dtype_include=['int64','float64'])),
        ('cat', cat_transformer, ['gender','Age_Group','wealth_segment','owns_car','state','job_title','job_industry_category']),
        #('obj', obj_transformer, ['job_title','job_industry_category'])
        ])

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=0))])

clf.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BC214FCE80>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender', 'Age_Group',
  

In [12]:
prediction = clf.predict(X_test)

In [13]:
len(prediction)

1000

In [14]:
df2['last_name'] = df2['last_name'].fillna('--')

In [25]:
def customer_rank(pred):
    if(pred == "Platinum Customer"):
        rank = 1
    elif(pred == "Very Loyal"):
        rank = 2
    elif(pred == "Becoming Loyal"):
        rank = 3
    elif(pred == "Recent Customer"):
        rank = 4
    elif(pred == "Potential Customer"):
        rank = 5
    elif(pred == "Late Bloomer"):
        rank = 6
    elif(pred == "Losing Customer"):
        rank = 7
    elif(pred == "High Risk Customer"):
        rank = 8
    elif(pred == "Almost Lost Customer"):
        rank = 9
    elif(pred == "Evasive Customer"):
        rank = 10
    elif(pred == "Lost Customer"):
        rank = 11
    return rank    

In [26]:
df = pd.DataFrame(columns=['Id','first_name','last_name','Customer Segment', 'Customer Rank'])
i=0
for pred in prediction:
    #print(str(i+1) + ' ' + df2['first_name'][i] + ' ' + df2['last_name'][i] + ' ' + pred)
    rank = customer_rank(pred)
    df = df.append({'Id': i+1, 'first_name': df2['first_name'][i], 'last_name': df2['last_name'][i], 'Customer Segment': pred, 'Customer Rank': rank}, ignore_index=True)
    i = i + 1

In [30]:
df = df.sort_values(by=['Customer Rank'])
df

,Id,first_name,last_name,Customer Segment,Customer Rank
0,1,Chickie,Brister,Platinum Customer,1
530,531,Dallas,Lavalde,Platinum Customer,1
586,587,Raynard,--,Platinum Customer,1
229,230,Abigale,Sives,Platinum Customer,1
884,885,Raleigh,Pont,Platinum Customer,1
...,...,...,...,...,...
556,557,Worthington,Ahmed,Lost Customer,11
825,826,Tessa,Heakey,Lost Customer,11
250,251,Sunny,Christescu,Lost Customer,11
152,153,Archibald,Blessed,Lost Customer,11


In [32]:
df.to_excel('D:/KPMG/final_result.xlsx', index=False)